In [58]:
!pip install transformers

In [59]:
!pip install selenium
!pip install webdriver-manager

In [60]:
!pip install langchain

In [61]:
!pip install langchain_nvidia_ai_endpoints

In [62]:
!pip install python-dotenv

In [101]:
!pip install selenium

In [63]:
def 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# Configure Selenium to connect to the open Chrome session
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
 
# Initialize WebDriver with the debugger connection
driver = webdriver.Chrome(options=chrome_options)

# Now `driver` controls your existing browser session
print(driver.title)  # Example action to verify connection


Google


In [ ]:
# def insert_js(driver):
#     elements = driver.execute_script("""
#     let elements = [];
#     document.querySelectorAll("a, button, input, textarea,li").forEach((el, index) => {
#         // Check if the element is visible
#         const style = window.getComputedStyle(el);
#         const isVisible = style.display !== 'none' && style.visibility !== 'hidden' && el.offsetHeight > 0 && el.offsetWidth > 0;
        
#         // If the element is visible, proceed
#         if (isVisible) {
#             // Generate a unique identifier
#             let uniqueId = el.tagName + '_' + index;
#             el.setAttribute('data-uid', uniqueId);

#             // Collect element details
#             elements.push({
#                 uid: uniqueId, // Unique identifier
#                 tag: el.tagName,
#                 text: el.innerText.trim() || null,
#                 title:el.title,
#                 placeholder: el.placeholder || null,
#                 name: el.name || null,
#                 href: el.href || null,
#                 type: el.type || null,
#                 value: el.value || null
#             });
#         }
#     });
#     return elements;
# """)

#     return elements

In [ ]:
# def insert_js(driver):
#     elements = driver.execute_script("""
#       let elements = [];
#     document.querySelectorAll("a, button, input, textarea, li, div").forEach((el, index) => {
#         // Check if the element is visible
#         const style = window.getComputedStyle(el);
#         const isVisible = style.display !== 'none' && style.visibility !== 'hidden' && el.offsetHeight > 0 && el.offsetWidth > 0;

#         // If the element is visible, proceed
#         if (isVisible) {
#             // For div elements, check for a click event listener
#             if (el.tagName === 'DIV') {
#                 // Check if the element has click event listeners or inline onclick
#                 if (!(
#                     (typeof getEventListeners === 'function' && getEventListeners(el).click) || 
#                     el.getAttribute('onclick') !== null
#                 )) {
#                     return; // Skip div if no click event
#                 }
#             }

#             // Generate a unique identifier
#             let uniqueId = el.tagName + '_' + index;
#             el.setAttribute('data-uid', uniqueId);

#             // Collect element details
#             elements.push({
#                 uid: uniqueId, // Unique identifier
#                 tag: el.tagName,
#                 text: el.innerText.trim() || null,
#                 title: el.title,
#                 placeholder: el.placeholder || null,
#                 name: el.name || null,
#                 href: el.href || null,
#                 type: el.type || null,
#                 value: el.value || null
#             });
#         }
#     });
#     return elements;
# """)

#     return elements

In [64]:
def insert_js(driver):
    elements = driver.execute_script("""
      let elements = [];
      document.querySelectorAll("a, button, input, textarea, div").forEach((el, index) => {
          // Check if the element is visible
          const style = window.getComputedStyle(el);
          const isVisible = style.display !== 'none' && style.visibility !== 'hidden' && el.offsetHeight > 0 && el.offsetWidth > 0;

          // If the element is visible, proceed
          if (isVisible) {
              // For div elements, check for click event listeners or cursor-pointer class
              if (el.tagName === 'DIV') {
                  const hasCursorPointer = el.classList.contains('cursor-pointer');
                  const hasClickListener = (typeof getEventListeners === 'function' && getEventListeners(el).click) || 
                                            el.getAttribute('onclick') !== null;

                  // Skip div if it doesn't have cursor-pointer or a click event listener
                  if (!(hasCursorPointer || hasClickListener)) {
                      return;
                  }
              }

              // Generate a unique identifier
              let uniqueId = el.tagName + '_' + index;
              el.setAttribute('data-uid', uniqueId);

              // Collect element details
              elements.push({
                  uid: uniqueId, // Unique identifier
                  tag: el.tagName,
                  text: el.innerText.trim() || null,
                  title: el.title || null,
                  placeholder: el.placeholder || null,
                  name: el.name || null,
                  href: el.href || null,
                  type: el.type || null,
                  value: el.value || null,
                  className: el.className || null,
              });
          }
      });
      return elements;
    """)

    return elements


In [65]:
def filter_elements(elements):
    filtered_elements = []
    for element in elements:
        filtered_element = {}
        for key, value in element.items():
            # Skip empty or None values
            if value in [None, ''] or key == 'className'  :
                continue

             # Special case: If the element is an <A> tag with no text or href
            if key == 'tag' and value == 'A':
                # Check if both text and href are missing
                if not element.get('text') or not element.get('href'):
                    filtered_element = 0
                    break
            # If the value is a URL, truncate it to the first two words
            if key == 'href' and isinstance(value, str):
                # truncated_url = " ".join(value.split("/")[2:3]) + "..."  # Extract first two words of the URL
                # filtered_element[key] = truncated_url
                pass
            else:
                filtered_element[key] = value
          # Only append non-empty filtered elements
        if filtered_element:
            filtered_elements.append(filtered_element)
    return filtered_elements

In [66]:
from transformers import GPT2Tokenizer

# Initialize tokenizer (use the correct tokenizer for your model)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def organize_elements(elements):
    categories = {
        "Links": [],
        "Text Areas": [],
        "Buttons": [],
        
    }
    max_tokens = 2000

    total_tokens = 0
    links = []

    # Categorize elements and count tokens for links
    for element in elements:
        if element["tag"] == "A":
            # Create ele as a list
            if(total_tokens < max_tokens):
                ele = [element["text"], element["uid"]]
                # Join the list elements into a single string for tokenization
                combined_text = " ".join(ele)
                token_count = len(tokenizer.encode(combined_text))
                total_tokens += token_count
                categories["Links"].append(ele)
        elif element["tag"] == "TEXTAREA" or (element["tag"] == "INPUT" and element.get("type") != "submit"):
            # Store only available fields: name, uid, title, placeholder
            if element.get("name") or element.get("title") or element.get("value") or element.get("placeholder"):
                text_area_entry = {
                    "name": element.get("name"),
                    "uid": element.get("uid"),
                    "title": element.get("title"),
                    "value":element.get("value"),
                    "placeholder": element.get("placeholder")
                }
                # Only add fields that are not None
                filtered_text_area = {key: value for key, value in text_area_entry.items() if value is not None}
                categories["Text Areas"].append(filtered_text_area)
        elif element["tag"] == "BUTTON" or (element["tag"] == "INPUT" and element.get("type") == "submit"):
            if element.get("name") or element.get("text") or element.get("value"):
                button_area_entry = {
                    "name": element.get("name"),
                    "text": element.get("text") or element.get("value"), 
                    "uid": element.get("uid")
                }
                # Filter out fields that are None
                filtered_button_area = {key: value for key, value in button_area_entry.items() if value is not None}
                categories["Buttons"].append(filtered_button_area)
        # elif element['tag'] == 'LI':
        #     if element.get("name") or element.get("text") or element.get("value"):
        #         li_area_entry = {
        #             "name": element.get("name"),
        #             "text": element.get("text") or element.get("value"), 
        #             "uid": element.get("uid")
        #         }
        #         # Filter out fields that are None
        #         filtered_li_area = {key: value for key, value in li_area_entry.items() if value is not None}
        #         categories["Lists"].append(filtered_li_area)
        elif element['tag'] == 'DIV':
            if element.get("name") or element.get("text") or element.get("value"):
                div_area_entry = [
                    element.get("text") or element.get("value"), 
                    element.get("uid")
                ]
                # Filter out fields that are None
                # filtered_div_area = {key: value for key, value in div_area_entry.items() if value is not None}
                categories["Links"].append(div_area_entry)

    # Update categories with valid links
    categories["Links"].append([element for element, _ in links])
    
    return categories

E:\RK\PROJECTS\webcontrol\webcon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
# def organize_elements(elements):
#     max_tokens = 2000
#     total_tokens = 0
#     structured_elements = []
#     links = []

#     # Process elements sequentially
#     for element in elements:
#         if element["tag"] == "A":
#             # Create ele as a list
#             ele = [element["text"], element["uid"]]
#             # Join the list elements into a single string for tokenization
#             combined_text = " ".join(ele)
#             token_count = len(tokenizer.encode(combined_text))
#             total_tokens += token_count
            
#             # Check if adding this link exceeds the token limit
#             if total_tokens <= max_tokens:
#                 structured_elements.append(ele)

#         elif element["tag"] == "TEXTAREA" or (element["tag"] == "INPUT" and element.get("type") != "submit"):
#             # Store only available fields: name, uid, title, placeholder
#             if element.get("name") or element.get("title") or element.get("value") or element.get("placeholder"):
#                 text_area_entry = {
#                     "name": element.get("name"),
#                     "uid": element.get("uid"),
#                     "title": element.get("title"),
#                     "value": element.get("value"),
#                     "placeholder": element.get("placeholder")
#                 }
#                 # Only add fields that are not None
#                 filtered_text_area = {key: value for key, value in text_area_entry.items() if value is not None}
#                 structured_elements.append(filtered_text_area)

#         elif element["tag"] == "BUTTON" or (element["tag"] == "INPUT" and element.get("type") == "submit"):
#             if element.get("name") or element.get("text") or element.get("value"):
#                 button_area_entry = {
#                     "name": element.get("name"),
#                     "text": element.get("text") or element.get("value"), 
#                     "uid": element.get("uid")
#                 }
#                 # Filter out fields that are None
#                 filtered_button_area = {key: value for key, value in button_area_entry.items() if value is not None}
#                 structured_elements.append( filtered_button_area)
#         elif element['tag'] == 'LI':
#             if element.get("name") or element.get("text") or element.get("value"):
#                 li_area_entry = {
#                     "name": element.get("name"),
#                     "text": element.get("text") or element.get("value"), 
#                     "uid": element.get("uid")
#                 }
#                 # Filter out fields that are None
#                 filtered_li_area = {key: value for key, value in li_area_entry.items() if value is not None}
#                 structured_elements.append( filtered_li_area)

#     return structured_elements

In [67]:
def fetch_html_content(driver):
    elements = insert_js(driver)
    filtered_elements = filter_elements(elements)
    stored_results=organize_elements(filtered_elements)
    # Display results with descriptive headings
    for category, items in stored_results.items():
        print(f"--- {category} ---")
        for item in items:
            print(item)
        print("\n")
    return stored_results

In [68]:
stored_results = fetch_html_content(driver)

# print(stored_results)

--- Links ---
['About', 'A_2']
['Store', 'A_3']
['Gmail', 'A_8']
['Images', 'A_10']
['Sign in', 'A_15']
['हिन्दी', 'A_146']
['বাংলা', 'A_147']
['తెలుగు', 'A_148']
['मराठी', 'A_149']
['தமிழ்', 'A_150']
['ગુજરાતી', 'A_151']
['ಕನ್ನಡ', 'A_152']
['മലയാളം', 'A_153']
['ਪੰਜਾਬੀ', 'A_154']
['Advertising', 'A_161']
['Business', 'A_162']
['How Search works', 'A_163']
['Privacy', 'A_165']
['Terms', 'A_166']
[]


--- Text Areas ---
{'name': 'q', 'uid': 'TEXTAREA_40', 'title': 'Search'}


--- Buttons ---
{'name': 'btnK', 'text': 'Google Search', 'uid': 'INPUT_123'}
{'name': 'btnI', 'text': "I'm Feeling Lucky", 'uid': 'INPUT_124'}




In [ ]:
# from selenium.webdriver.common.by import By

In [ ]:

# # Find all elements with the class `cursor-pointer`
# cursor_pointer_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'cursor-pointer')]")

# # Store details of the elements
# elements_details = []
# for element in cursor_pointer_elements:
#     elements_details.append({
#         "tag": element.tag_name,
#         "id": element.get_attribute('id'),
#         "text": element.text,
       
#     })

# # Print the collected elements
# print("Elements with 'cursor-pointer':")
# for detail in elements_details:
#     print(f"Tag: {detail['tag']}, ID: {detail['id']}, Text: {detail['text']}")
#     print("-" * 50)

In [69]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [70]:

# Define functions to perform actions based on model response
def click_element(driver, tag=None, text=None, data_uid=None):
    try:
        if data_uid:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-uid='{data_uid}']"))
            )
        elif tag and text:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f"//{tag}[text()='{text}']"))
            )
        else:
            raise ValueError("Either data_uid or both tag and text must be provided.")

        # Scroll into view and use JavaScript to click
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
        time.sleep(0.5)  # Small delay
        driver.execute_script("arguments[0].click();", element)
        print(f"Clicked on element with tag '{tag}' or data-uid '{data_uid}'")
    except Exception as e:
        print(f"Failed to click on element with tag '{tag}' or data-uid '{data_uid}': {e}")

def enter_text(driver, tag=None, text_to_enter=None, data_uid=None):
    try:
        # Find element by data-uid if available
        if data_uid:
            element = driver.find_element(By.CSS_SELECTOR, f"[data-uid='{data_uid}']")
        elif tag:
            element = driver.find_element(By.TAG_NAME, tag)
        else:
            raise ValueError("Either data_uid or tag must be provided.")

        element.clear()  # Clear any pre-existing text
        element.send_keys(text_to_enter)  # Enter the text
        element.send_keys(Keys.RETURN)  # Press Enter
        print(f"Entered '{text_to_enter}' in element with tag '{tag}' or data-uid '{data_uid}'")
    except Exception as e:
        print(f"Failed to enter text '{text_to_enter}' in element with tag '{tag}' or data-uid '{data_uid}': {e}")

# Sample model responses
# model_response_click = "1,'DIV_312'"
# model_response_enter_text = "2, Technical Guruji, TEXTAREA_4"

# Function to interpret and perform action
def perform_action(driver, model_response):
    # Match model response with regex patterns
    click_match = re.match(r"1, ([\w\d_]+)", model_response)
    enter_match = re.match(r"2,(.+), ([\w\d_]+)", model_response)

    if click_match:
        # Extract data-uid for click action
        data_uid = click_match.group(1)
        click_element(driver, data_uid=data_uid)
    elif enter_match:
        # Extract text to enter and data-uid for text entry
        text_to_enter = enter_match.group(1).strip()
        data_uid = enter_match.group(2)
        enter_text(driver, text_to_enter=text_to_enter, data_uid=data_uid)
    else:
        print("Invalid model response format.")

# Example usage
# perform_action(driver, model_response_click)
# perform_action(driver, model_response_enter_text)

In [71]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain import PromptTemplate, LLMChain
from langchain_core.output_parsers import StrOutputParser
import getpass
import os

In [72]:
from dotenv import load_dotenv

In [73]:
load_dotenv(dotenv_path="apikey.env")

# Now access your variables
print(os.getenv("LANGCHAIN_API_KEY"))        # should print: 86accbf
print(os.getenv("NVIDIA_API_KEY"))           # should print: my key
print(os.getenv("LANGCHAIN_TRACING_V2")) 


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
true


In [74]:
# Access variables
print(os.getenv("LANGCHAIN_API_KEY"))

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
os.environ["NVIDIA_API_KEY"] = getpass.getpass()

In [ ]:
os.environ["NVIDIA_API_KEY"]

In [ ]:
os.environ["LANGCHAIN_API_KEY"]

In [ ]:
os.environ

In [75]:
model = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [76]:
parser = StrOutputParser()

In [77]:
# Define the updated prompt template
prompt_template = """
**Context:** 
you are a human using computer, By seeing the HTML content like links, buttons and text area present on any page provided below you can person actions to acieve goal

**Goal:** 
Navigate to {user_prompt}

**Memory:** 
below are the Previously visited pages and actions performed by you, avoid repetition:
{memory_data}

**Current Page:** 
Page title: {current_page}

**Html content:** 
 ----links-----
 {links}
 ----Text Area------
 {text}
 -----Buttons------
 {buttons}



HTML Content format:
- Links: (text, uid)
- Text Areas: (fields with uid)
- Buttons: (fields with uid)


**Instructions:** 
Based on the goal and HTML content, guide navigation or interaction by responding in the exact format below. Follow these rules strictly.

- To click a button or link: `"1, (uid)"`
- To enter text in a text area: `"2, (text to enter), (uid)"`
- If the goal is achieved or no further actions are required: `"done"`

**Response Rules:** 
1. Always include `uid` in the response.
2. look into Memory and never repeart action, just output 'done'
3. Respond with only one action per reply.
4. [important] Do not provide explanations, reasoning, or text outside the required format.

**Reminder**
- DO NOT REPEAT STEP THAT IS ALREADY DONE ON MEMEORY
"""

In [78]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["user_prompt", "links","text","buttons","current_page","memory_data"],
)

# partial_prompt = prompt.partial(html_content=fetch_html_content(driver),current_page=get_page_description(driver))

In [103]:
formatted_prompt = prompt_template.format(user_prompt="show me technical guruji channel on youtube",
                                          links=stored_results["Links"],
                                          text= stored_results["Text Areas"],
                                          buttons = stored_results["Buttons"],
                                          current_page=get_page_description(driver),
                                          memory_data="")
print(formatted_prompt)

KeyError: 'Html_content'

In [80]:
def get_page_description(driver):
    title = driver.title
    return title

In [81]:
def browser_automation_workflow(driver):
 # Fetch necessary data
    stored_results = fetch_html_content(driver)
    current_page = get_page_description(driver)
    
    # Prepare the prompt
    partial_prompt = prompt.partial(
        links=stored_results["Links"],
        text= stored_results["Text Areas"],
        buttons = stored_results["Buttons"],
        current_page=get_page_description(driver),
        memory_data=memory_data
    )
    chain = partial_prompt | model | parser
    # Combine prompt with the model chain
    print(memory_data)
    # Invoke the chain with user intent
    response = chain.invoke({"user_prompt": "go to irctc website and show me train on 30th dec 2024 from mumbai to jaipur"})

    # Debug: Print the response for monitoring
    print(f"Model Response: {response}")
    page_entry = {"page_title": current_page, "actions":response.strip() }
    memory_data.append(page_entry)
    # Perform the action on the browser based on the response
    perform_action(driver, response)

    return response

In [82]:
memory_data = []
while True:
    action = browser_automation_workflow(driver)
    if "done" in action:
        break;

--- Links ---
['About', 'A_2']
['Store', 'A_3']
['Gmail', 'A_8']
['Images', 'A_10']
['Sign in', 'A_15']
['हिन्दी', 'A_146']
['বাংলা', 'A_147']
['తెలుగు', 'A_148']
['मराठी', 'A_149']
['தமிழ்', 'A_150']
['ગુજરાતી', 'A_151']
['ಕನ್ನಡ', 'A_152']
['മലയാളം', 'A_153']
['ਪੰਜਾਬੀ', 'A_154']
['Advertising', 'A_161']
['Business', 'A_162']
['How Search works', 'A_163']
['Privacy', 'A_165']
['Terms', 'A_166']
[]


--- Text Areas ---
{'name': 'q', 'uid': 'TEXTAREA_40', 'title': 'Search'}


--- Buttons ---
{'name': 'btnK', 'text': 'Google Search', 'uid': 'INPUT_123'}
{'name': 'btnI', 'text': "I'm Feeling Lucky", 'uid': 'INPUT_124'}


[]


E:\RK\PROJECTS\webcontrol\webcon\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=bb3b8df8-4150-43b2-995e-987acb68e88a; trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=32714742-efab-499a-a81f-f025b3a19dd3; trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=76d243ac-1c1d-43fb-90f1-7ba032f1bf97


Model Response: 2, irctc, TEXTAREA_40
Entered 'irctc' in element with tag 'None' or data-uid 'TEXTAREA_40'
--- Links ---
['About', 'A_2']
['Store', 'A_3']
['Gmail', 'A_8']
['Images', 'A_10']
['Sign in', 'A_15']
['हिन्दी', 'A_151']
['বাংলা', 'A_152']
['తెలుగు', 'A_153']
['मराठी', 'A_154']
['தமிழ்', 'A_155']
['ગુજરાતી', 'A_156']
['ಕನ್ನಡ', 'A_157']
['മലയാളം', 'A_158']
['ਪੰਜਾਬੀ', 'A_159']
['Advertising', 'A_166']
['Business', 'A_167']
['How Search works', 'A_168']
['Privacy', 'A_170']
['Terms', 'A_171']
[]


--- Text Areas ---
{'name': 'q', 'uid': 'TEXTAREA_40', 'title': 'Search', 'value': 'irctc'}


--- Buttons ---
{'name': 'btnK', 'text': 'Google Search', 'uid': 'INPUT_123'}
{'name': 'btnI', 'text': "I'm Feeling Lucky", 'uid': 'INPUT_124'}


[{'page_title': 'Google', 'actions': '2, irctc, TEXTAREA_40'}]
Model Response: 1, INPUT_123


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=76d243ac-1c1d-43fb-90f1-7ba032f1bf97; trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=f72ba7c7-a3aa-43e0-a6cd-dc9e2948719c; trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=f72ba7c7-a3aa-43e0-a6cd-dc9e2948719c; trace=bb3b8df8-4150-43b2-995e-987acb68e88a,id=bb3b8df8-4150-43b2-995e-987acb68e88a; trace=1107e69b-15f7-4868-b418-a872915a8fbf,id=1107e69b-15f7-4868-b418-a872915a8fbf; trace=1107e69b-15f7-4868-b418-a872915a8fbf,id=cd33915c-2fe4-46ae-b73b-888013469781; trace=1107e69b-15f7-4868-b418-a872915a8fbf,id=cd33915c-2fe4-46ae-b73b-888013469781; trace=1107e69b-15f7-4868-b418-a872915a8fbf,id=00611320-56b0-46a8-b32d-ffa121a0376b; trace=1107e69b-15f7-4868-b418-a872915a8fbf,id

Failed to click on element with tag 'None' or data-uid 'INPUT_123': Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7e824fe75+79173]
	GetHandleVerifier [0x0x7ff7e824fed0+79264]
	(No symbol) [0x0x7ff7e8009e5a]
	(No symbol) [0x0x7ff7e8060586]
	(No symbol) [0x0x7ff7e806083c]
	(No symbol) [0x0x7ff7e80b4247]
	(No symbol) [0x0x7ff7e80889af]
	(No symbol) [0x0x7ff7e80b100d]
	(No symbol) [0x0x7ff7e8088743]
	(No symbol) [0x0x7ff7e80514c1]
	(No symbol) [0x0x7ff7e8052253]
	GetHandleVerifier [0x0x7ff7e851a2ad+3004797]
	GetHandleVerifier [0x0x7ff7e85146fd+2981325]
	GetHandleVerifier [0x0x7ff7e8533350+3107360]
	GetHandleVerifier [0x0x7ff7e826a9fe+188622]
	GetHandleVerifier [0x0x7ff7e827228f+219487]
	GetHandleVerifier [0x0x7ff7e8258dc4+115860]
	GetHandleVerifier [0x0x7ff7e8258f79+116297]
	GetHandleVerifier [0x0x7ff7e823f528+11256]
	BaseThreadInitThunk [0x0x7ff8e9207374+20]
	RtlUserThreadStart [0x0x7ff8e973cc91+33]

--- Links ---
['Accessibility help', 'A_3']
['Sign in', 'A_168']
['News', 'A_195']
['I

...

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=92dea3b1-1934-4ee4-b35d-50c2425ca2ef; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=242d0930-404c-4c18-ae9e-4a2ff3e10141; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=242d0930-404c-4c18-ae9e-4a2ff3e10141; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=08661d61-0b76-47c3-8e97-90ba9def9b18; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=08661d61-0b76-47c3-8e97-90ba9def9b18; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=7b2818dd-5c73-4f9e-a17a-322e8578630f; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=7b2818dd-5c73-4f9e-a17a-322e8578630f; trace=92dea3b1-1934-4ee4-b35d-50c2425ca2ef,id=92dea3b1-1934-4ee4-b35d-50c2425ca2ef


Model Response: 1, BUTTON_346


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=eb364e5b-9709-4e52-8f22-77bcbd71c7b5; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=a0adc657-31fb-4941-a189-c44a446ca579; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=a0adc657-31fb-4941-a189-c44a446ca579; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=020bd2fc-22d1-46b6-acbf-2e806773323a; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=020bd2fc-22d1-46b6-acbf-2e806773323a; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=17709091-03d5-424c-a6c5-ba9fc1d8b76e; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=17709091-03d5-424c-a6c5-ba9fc1d8b76e; trace=eb364e5b-9709-4e52-8f22-77bcbd71c7b5,id=eb364e5b-9709-4e52-8f22-77bcbd71c7b5


Clicked on element with tag 'None' or data-uid 'BUTTON_346'
--- Links ---
['PNR STATUS', 'A_290']
['CHARTS / VACANCY', 'A_292']
['https://equery.irctc.co.in', 'A_361']
['BEWARE OF FRAUDSTERS: Always download official IRCTC Rail Connect App from the Google Play Store or Apple App Store only.', 'A_362']
['FLIGHTS', 'A_379']
['HOTELS', 'A_380']
['RAIL DRISHTI', 'A_381']
['E-CATERING', 'A_382']
['BUS', 'A_383']
['HOLIDAY PACKAGES', 'A_384']
['TOURIST TRAIN', 'A_385']
['HILL RAILWAYS', 'A_386']
['CHARTER TRAIN', 'A_387']
['GALLERY', 'A_388']
['Read More', 'A_426']
['Read More', 'A_431']
['Read More', 'A_436']
['Read More', 'A_441']
['Read More', 'A_446']
['Help & Support', 'A_502']
['CRIS', 'A_510']
['Compatible Browsers', 'A_511']
[]


--- Text Areas ---
{'uid': 'INPUT_305', 'value': '30-12-2024'}
{'name': 'concessionBooking', 'uid': 'INPUT_346', 'value': 'on'}
{'name': 'dateSpecific', 'uid': 'INPUT_347', 'value': 'on'}
{'name': 'availableBerth', 'uid': 'INPUT_348', 'value': 'on'}
{'name':

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=e446fb15-696d-4c9f-901f-51cf7f6d240f; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=a57ba2a5-0b2d-4a0f-ba40-e134b58f56d3; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=a57ba2a5-0b2d-4a0f-ba40-e134b58f56d3; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=47d00cd2-0e09-44ca-9e0d-0762cc49dbee; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=47d00cd2-0e09-44ca-9e0d-0762cc49dbee; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=c91c9a11-a822-4e9c-8a63-9cc361c9d3ff; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=c91c9a11-a822-4e9c-8a63-9cc361c9d3ff; trace=e446fb15-696d-4c9f-901f-51cf7f6d240f,id=e446fb15-696d-4c9f-901f-51cf7f6d240f
Failed to send compressed multipart ingest: la

In [ ]:
# alternate

In [83]:
# Define the updated prompt template
prompt_template = """
**Context:** 
Identify the current webpage based on the provided HTML content and avoid repeating actions from memory.

**Goal:** 
Navigate to {user_prompt}

**Memory:** 
Previously visited pages and actions performed to avoid repetition:
{memory_data}

**Current Page:** 
Page title: {current_page}

**Html content:** 
{Html_content}

HTML Content format:
- Links: (text, uid)
- Text Areas: (fields with uid)
- Buttons: (fields with uid)

**Instructions:** 
Based on the goal and HTML content, guide navigation or interaction by responding in the exact format below. Follow these rules strictly.

- To click a button or link: `"1, (uid)"`
- To enter text in a text area: `"2, (text to enter), (uid)"`
- If the goal is achieved or no further actions are required: `"done"`

**Response Rules:** 
1. Always include `uid` in the response.
2. look into Memory and never repeart action, just output 'done'
3. Respond with only one action per reply.
4. Do not provide explanations, reasoning, or text outside the required format.
"""

In [84]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["user_prompt","Html_content","current_page","memory_data"],
)

# partial_prompt = prompt.partial(html_content=fetch_html_content(driver),current_page=get_page_description(driver))

In [96]:
formatted_prompt = prompt_template.format(user_prompt="show me technical guruji channel on youtube",Html_content=stored_results,current_page=get_page_description(driver),
        memory_data="")
print(formatted_prompt)


**Context:** 
Identify the current webpage based on the provided HTML content and avoid repeating actions from memory.

**Goal:** 
Navigate to show me technical guruji channel on youtube

**Memory:** 
Previously visited pages and actions performed to avoid repetition:


**Current Page:** 
Page title: youtube - Google Search

**Html content:** 
{'Links': [['About', 'A_2'], ['Store', 'A_3'], ['Gmail', 'A_8'], ['Images', 'A_10'], ['Sign in', 'A_15'], ['हिन्दी', 'A_146'], ['বাংলা', 'A_147'], ['తెలుగు', 'A_148'], ['मराठी', 'A_149'], ['தமிழ்', 'A_150'], ['ગુજરાતી', 'A_151'], ['ಕನ್ನಡ', 'A_152'], ['മലയാളം', 'A_153'], ['ਪੰਜਾਬੀ', 'A_154'], ['Advertising', 'A_161'], ['Business', 'A_162'], ['How Search works', 'A_163'], ['Privacy', 'A_165'], ['Terms', 'A_166'], []], 'Text Areas': [{'name': 'q', 'uid': 'TEXTAREA_40', 'title': 'Search'}], 'Buttons': [{'name': 'btnK', 'text': 'Google Search', 'uid': 'INPUT_123'}, {'name': 'btnI', 'text': "I'm Feeling Lucky", 'uid': 'INPUT_124'}]}

HTML Content forma

In [99]:
def browser_automation_workflow(driver):
 # Fetch necessary data
    stored_results = fetch_html_content(driver)
    current_page = get_page_description(driver)
    
    # Prepare the prompt
    partial_prompt = prompt.partial(
        Html_content=stored_results,
        current_page=get_page_description(driver),
        memory_data=memory_data
    )
    chain = partial_prompt | model | parser
    # Combine prompt with the model chain
    print(memory_data)
    # Invoke the chain with user intent
    response = chain.invoke({"user_prompt": "Play quala song on youtube"})

    # Debug: Print the response for monitoring
    print(f"Model Response: {response}")
    page_entry = {"page_title": current_page, "actions":response.strip() }
    memory_data.append(page_entry)
    # Perform the action on the browser based on the response
    perform_action(driver, response)

    return response

In [100]:
memory_data = []
while True:
    action = browser_automation_workflow(driver)
    if "done" in action:
        break;

--- Links ---
['About', 'A_2']
['Store', 'A_3']
['Gmail', 'A_8']
['Images', 'A_10']
['Sign in', 'A_15']
['हिन्दी', 'A_140']
['বাংলা', 'A_141']
['తెలుగు', 'A_142']
['मराठी', 'A_143']
['தமிழ்', 'A_144']
['ગુજરાતી', 'A_145']
['ಕನ್ನಡ', 'A_146']
['മലയാളം', 'A_147']
['ਪੰਜਾਬੀ', 'A_148']
['Advertising', 'A_155']
['Business', 'A_156']
['How Search works', 'A_157']
['Privacy', 'A_159']
['Terms', 'A_160']
[]


--- Text Areas ---
{'name': 'q', 'uid': 'TEXTAREA_40', 'title': 'Search'}


--- Buttons ---
{'name': 'btnK', 'text': 'Google Search', 'uid': 'INPUT_123'}
{'name': 'btnI', 'text': "I'm Feeling Lucky", 'uid': 'INPUT_124'}


[]
Model Response: 2, quala song, TEXTAREA_40
Entered 'quala song' in element with tag 'None' or data-uid 'TEXTAREA_40'
--- Links ---
['About', 'A_2']
['Store', 'A_3']
['Gmail', 'A_8']
['Images', 'A_10']
['Sign in', 'A_15']
['हिन्दी', 'A_145']
['বাংলা', 'A_146']
['తెలుగు', 'A_147']
['मराठी', 'A_148']
['தமிழ்', 'A_149']
['ગુજરાતી', 'A_150']
['ಕನ್ನಡ', 'A_151']
['മലയാളം', 'A_

In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import threading

# Global memory for interaction history
memory_data = []

def main():
    # Setup Selenium driver
    options = Options()
    options.add_experimental_option("debuggerAddress", "localhost:9222")
    driver = webdriver.Chrome(options=options)

    print("=== AI Browser Assistant ===")
    print("Type your command and hit ENTER.")
    print("Type 'stop' anytime to exit.\n")

    while True:
        user_input = input(">>> ")

        if user_input.strip().lower() == "stop":
            print("Manual stop requested.")
            break

        action = browser_automation_workflow(driver, user_input)

        if "done" in action.lower():
            print("Workflow indicated completion.")
            break

    driver.quit()

# Modify your function to accept user prompt
def browser_automation_workflow(driver, user_prompt):
    stored_results = fetch_html_content(driver)
    current_page = get_page_description(driver)

    partial_prompt = prompt.partial(
        Html_content=stored_results,
        current_page=current_page,
        memory_data=memory_data
    )
    chain = partial_prompt | model | parser

    response = chain.invoke({"user_prompt": user_prompt})

    print(f"Model Response: {response}")
    page_entry = {"page_title": current_page, "actions": response.strip()}
    memory_data.append(page_entry)

    perform_action(driver, response)

    return response
